In [37]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
search = TavilySearch(max_results=5)

In [ ]:

search_results = search.invoke("What is the weather in Berlin")
print(search_results)

{'query': 'What is the weather in Berlin', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Berlin', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Berlin', 'region': 'Berlin', 'country': 'Germany', 'lat': 52.5167, 'lon': 13.4, 'tz_id': 'Europe/Berlin', 'localtime_epoch': 1751406917, 'localtime': '2025-07-01 23:55'}, 'current': {'last_updated_epoch': 1751406300, 'last_updated': '2025-07-01 23:45', 'temp_c': 22.3, 'temp_f': 72.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 120, 'wind_dir': 'ESE', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 46, 'cloud': 0, 'feelslike_c': 24.0, 'feelslike_f': 75.2, 'windchill_c': 23.3, 'windchill_f': 73.9, 'heatindex_c': 24.4, 'heatindex_f': 75.8, 'dewpoint_c': 8.3, 'dewpoint_f': 46.9, 'vis_km': 10.0, 'vis_miles': 6

In [17]:
tools = [search]

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o", model_provider="openai")

In [18]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I assist you today?'

In [21]:
model_with_tools = model.bind_tools(tools)

In [22]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I assist you today?

Tool calls: []


In [27]:
query = "Search for the weather in Berlin"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Berlin'}, 'id': 'call_qush0UNfy5msZAiq7oIXB5B6', 'type': 'tool_call'}]


In [40]:
agent_executor = create_react_agent(model, tools)

In [32]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [43]:
system_massage = {"role": "system", "content": "Before providing the answer of the users question, please check that users query is fully and accurately answered if not then repeat until it is"}
input_message = {"role": "user", "content": "What is the actual currency rate for the euro to the dollar?"}
response = agent_executor.invoke({"messages": [system_massage, input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ System Message ================================

Before providing the answer of the users question, please check that users query is fully and accurately answered if not then repeat until it is
================================ Human Message =================================

What is the actual currency rate for the euro to the dollar?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_tmjhmV8pedrWuvbVCiyZ0FwP)
 Call ID: call_tmjhmV8pedrWuvbVCiyZ0FwP
  Args:
    query: current EUR to USD exchange rate
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current EUR to USD exchange rate", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Live Euro to US Dollar Exchange Rate - € 1 EUR/USD Today", "url": "https://eur.currencyrate.today/usd", "content": "Get the latest and best €1 

In [45]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the actual currency rate for the euro to the dollar?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_qilvZt7rNlW6PerLXqyD8fha)
 Call ID: call_qilvZt7rNlW6PerLXqyD8fha
  Args:
    query: current EUR to USD exchange rate
    topic: finance
================================= Tool Message =================================
Name: tavily_search

{"query": "current EUR to USD exchange rate", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "EUR to USD Exchange Rate - The Wall Street Journal", "url": "https://www.wsj.com/market-data/quotes/fx/EURUSD", "content": "Current exchange rate for EUR to USD.Open 1.1788 Prior Close 1.1789 (06/30/25) 1 Day EURUSD -0.04% USDJPY -0.25% BUXX -0.04%", "score": 0.902687, "raw_content": null}, {"title": "EUR/USD (EUR-USD.F) live rate, chart and news - Yahoo Finance", "url": "